# SIMWE testing

In [ ]:
import os
import subprocess
import sys
import shutil

# Download zip
!curl https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.zip -o ~/data/nc_spm_08_grass7.zip

# Unpack zip to grassdata
shutil.unpack_archive(
    os.path.expanduser("~/data/nc_spm_08_grass7.zip"),
    os.path.expanduser("~/data/"),
    "zip",
)

In [ ]:
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)
# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/data/nc_spm_08_grass7")

In [ ]:
elevation = "elev_lid792_1m"
gs.run_command("g.region", raster=elevation)
gs.run_command("r.slope.aspect", elevation=elevation, dx="dx", dy="dy")
gs.run_command("g.region", s="s+290")

elevation_map = gj.Map(use_region=True)
elevation_map.d_rast(map=elevation)
elevation_map.show()

## No infiltration

In [ ]:
# Output time step increment
increment = 1
niterations = 10

gs.run_command(
    "r.sim.water",
    elevation=elevation,
    dx="dx",
    dy="dy",
    rain_value=50,
    infil_value=0,
    depth="depth",
    discharge="disch",
    niterations=niterations,
    output_step=increment,
    flags="t",
)

img = gj.Map()
img.d_rast(map=f"depth.{niterations}")
img.d_legend(raster=f"depth.{niterations}", at=(45, 95, 85, 90), flags="l")
img.show()

In [ ]:
img = gj.InteractiveMap(width=1000)
img.add_raster(f"depth.{niterations}")
img.add_raster(f"disch.{niterations}")
img.show()

In [ ]:
# Register the time series
gs.run_command(
    "t.create",
    output="depth",
    temporaltype="relative",
    title="Overland flow depth",
    description="Overland flow depth",
)
# Register the time series
maps = gs.list_strings(type="raster", pattern="depth*")
gs.run_command("t.register", input="depth", maps=maps)

# Create a time series map
flow_map = gj.TimeSeriesMap()

# Add the time series data
flow_map.add_raster_series("depth")

# Add map features
flow_map.d_legend(
    raster=f"depth.{niterations}",
    title="Depth [m]",
    flags="bt",
    border_color="none",
    at=(1, 50, 0, 5),
)

# Display the map
flow_map.show()

## Infiltration
Setup rainfall excess and infiltration layers:

In [ ]:
infil_upper = 0
infil_lower = 10

gs.mapcalc(f"infil = if(y() > 220520, {infil_upper}, {infil_lower})")

rainfall_upper = 50
rainfall_lower = 50

gs.mapcalc(f"rainfall = if(y() > 220520, {rainfall_upper}, {rainfall_lower})")

In [ ]:
increment = 1
niterations = 10

output_suffix = (
    f"rain_{rainfall_upper}_{rainfall_lower}_infil_{infil_upper}_{infil_lower}"
)

gs.run_command(
    "r.sim.water",
    elevation=elevation,
    dx="dx",
    dy="dy",
    rain="rainfall",
    infil="infil",
    depth="depth_" + output_suffix,
    discharge="disch_" + output_suffix,
    niterations=niterations,
    output_step=increment,
    flags="t",
)

img = gj.Map()
img.d_rast(map=f"depth_{output_suffix}.{niterations}")
img.d_legend(
    raster=f"depth_{output_suffix}.{niterations}", at=(45, 95, 85, 90), flags="l"
)
img.show()

In [ ]:
img = gj.InteractiveMap()
img.add_raster(f"depth_{output_suffix}.{niterations}")
img.add_raster(f"disch_{output_suffix}.{niterations}")
img.show()

In [ ]:
print(gs.read_command("r.info", map=f"depth_{output_suffix}.{niterations}"))